# 1부. 기초편 데이터 가공
## 02장. 대리점 데이터를 가공하는 테크닉 1제

## 📣 고객의 소리
우리 회사에서는 고객 정보를 엑셀로 관리합니다. 장사는 잘되고 대리점의 매출도 안정적입니다.

데이터가 풍부할 때 데이터 분석을 해두면 여러가지 발견을 할 수 있지 않을가 합니다.

시험 삼아 데이터 분석을 부탁드립니다.

<br>

## 📍 전제조건

해당 대리점에서는 상품 A ~ Z까지 26개의 상품을 취급하며, 매출 이력과 고객정보 데이터는 

담당사원이 시스템에 직접 입력한다. 집계 기간에 상품 단가의 변동은 없었고 매출 이력은 시스템에서

CSV 파일로 출력했습니다. 고객 정보는 대리점 관리자가 주별로 집계해서 엑셀로 관리한다.

In [96]:

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from IPython.core.display import display, HTML  # 화면크기 조절
display(HTML("<style>.container { width: 75% !important; }</style>"))

import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential

### 011. 데이터를 읽어 들이자

In [97]:

# 매출 이력 데이터
uriage_data = pd.read_csv("/Users/jeongjihwan/Downloads/pyda100-master/2장/uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [98]:

# 고객 정보 데이터
kokyaku_data = pd.read_excel("/Users/jeongjihwan/Downloads/pyda100-master/2장/kokyaku_daicho.xlsx")
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


* 수작업으로 작성된 데이터는 데이터의 정합성 문제 발생(입력 오류 및 포맷 차이 등)

### 012. 데이터의 오류를 살펴보자

In [99]:

uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

* 이렇게 되면 상품A와 상 품 a, 상품 a 모두 다른 상품으로 집계되어 잘못된 분석을 하게됨.

In [100]:

uriage_data["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

* 집계 대상 데이터에 오류 및 결측치가 있으면 정확한 집계가 불가능.

### 013. 데이터에 오류가 있는 상태로 일단 집계 해보자.

* 데이터의 오류가 집계에 어느 정도 영향을 미치는지 일단 확인

In [101]:

uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0)  # fill_value=0 -> NaN값은 0으로 나타냄
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


* 데이터프레임 결과에서 확인할 수 있듯이 상품s과 상품 S이 서로 다른 상품으로 집계되고 있음.


* 원래 상품은 26개인데 99개로 나타나고 있음.

In [102]:

# 가로축에 item_price를 설정
res = uriage_data.pivot_table(index="purchase_month",
                              columns="item_name",
                              values="item_price",
                              aggfunc="sum",
                              fill_value=0)  # fill_value=0 -> NaN값은 0으로 나타냄
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


* 결국 위의 결과도 모두 잘못된 것임.

### 014. 상품명 오류를 수정하자

In [103]:

print(len(pd.unique(uriage_data.item_name)))

99


In [104]:
uriage_data["item_name"]

0         상품A
1       상 품 S
2       상 품 a
3         상품Z
4         상품a
        ...  
2994      상품Y
2995      상품M
2996      상품Q
2997      상품H
2998      상품D
Name: item_name, Length: 2999, dtype: object

In [105]:

uriage_data["item_name"] = uriage_data["item_name"].str.upper()  # 영어 모두 대문자로
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")  # 문자열 공백 없애기
uriage_data.sort_values("item_name", ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [106]:
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")

In [107]:

# 검증
print(pd.unique(uriage_data.item_name))
print(len(pd.unique(uriage_data.item_name)))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


### 015. 금액의 결측치를 수정하자

In [108]:

uriage_data.isnull().any(axis=0)  # any() -> 하나라도 True가 있으면 True 출력 <=> all()은 모두 True 여야 True 출력

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

* item_price 칼럼에 결측치가 있다는걸 알 수 있음

In [109]:
fig_is_null

0       False
1        True
2        True
3       False
4        True
        ...  
2994    False
2995    False
2996     True
2997     True
2998    False
Name: item_price, Length: 2999, dtype: bool

In [110]:

fig_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[fig_is_null, "item_name"].unique()):  # price값이 Null인 고유 상품이름들
    price = uriage_data.loc[(~fig_is_null) & (uriage_data["item_name"] == trg), "item_price"].values[0] # ~은 NOT을 의미
    uriage_data['item_price'].loc[(fig_is_null) & (uriage_data["item_name"] == trg)] = price
    
    
uriage_data[["item_name", "item_price"]].head()

/opt/homebrew/Caskroom/miniforge/base/envs/tf/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,item_name,item_price
0,상품A,100.0
1,상품S,1900.0
2,상품A,100.0
3,상품Z,2600.0
4,상품A,100.0


* 결측 price가 있는 상품을 해당 상품에 대한 값이 올바르게 입력돼있는 price를 찾아 그 값으로 채워넣는 방식.

In [111]:

uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

* 방심하지 말고 검증은 무조건 진행한다.

In [112]:

for index, trg in enumerate(list(uriage_data["item_name"].sort_values().unique())):
    print(index, trg + "의 최고가 : " + str(uriage_data.loc[uriage_data["item_name"] == trg]["item_price"].max())
         + "의 최저가 : " + str(uriage_data.loc[uriage_data["item_name"] == trg]["item_price"].min(skipna=False)))

0 상품A의 최고가 : 100.0의 최저가 : 100.0
1 상품B의 최고가 : 200.0의 최저가 : 200.0
2 상품C의 최고가 : 300.0의 최저가 : 300.0
3 상품D의 최고가 : 400.0의 최저가 : 400.0
4 상품E의 최고가 : 500.0의 최저가 : 500.0
5 상품F의 최고가 : 600.0의 최저가 : 600.0
6 상품G의 최고가 : 700.0의 최저가 : 700.0
7 상품H의 최고가 : 800.0의 최저가 : 800.0
8 상품I의 최고가 : 900.0의 최저가 : 900.0
9 상품J의 최고가 : 1000.0의 최저가 : 1000.0
10 상품K의 최고가 : 1100.0의 최저가 : 1100.0
11 상품L의 최고가 : 1200.0의 최저가 : 1200.0
12 상품M의 최고가 : 1300.0의 최저가 : 1300.0
13 상품N의 최고가 : 1400.0의 최저가 : 1400.0
14 상품O의 최고가 : 1500.0의 최저가 : 1500.0
15 상품P의 최고가 : 1600.0의 최저가 : 1600.0
16 상품Q의 최고가 : 1700.0의 최저가 : 1700.0
17 상품R의 최고가 : 1800.0의 최저가 : 1800.0
18 상품S의 최고가 : 1900.0의 최저가 : 1900.0
19 상품T의 최고가 : 2000.0의 최저가 : 2000.0
20 상품U의 최고가 : 2100.0의 최저가 : 2100.0
21 상품V의 최고가 : 2200.0의 최저가 : 2200.0
22 상품W의 최고가 : 2300.0의 최저가 : 2300.0
23 상품X의 최고가 : 2400.0의 최저가 : 2400.0
24 상품Y의 최고가 : 2500.0의 최저가 : 2500.0
25 상품Z의 최고가 : 2600.0의 최저가 : 2600.0


* 각 상품의 금액이 정상적으로 수정됐는지 확인!
* skipna=True 옵션

기술 통계 함수에서는 skipna=True가 기본으로 설정 되어 있습니다.

만약, skipna=False로 설정하게 된다면, NaN 값이 있는 column은 NaN 값으로 출력 됩니다.

### 016. 고객 이름의 오류를 수정하자

In [113]:

kokyaku_data["고객이름"].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [114]:

uriage_data["customer_name"].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [115]:

kokyaku_data["고객이름"] = kokyaku_data["고객이름"].str.replace(" ", "")  # 공백 두칸도 제거
kokyaku_data["고객이름"] = kokyaku_data["고객이름"].str.replace(" ", "")  # 한칸 제거
kokyaku_data["고객이름"].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

###  017. 날짜 오류를 수정하자

In [116]:
kokyaku_data["등록일"]

0      2018-01-04 00:00:00
1                    42782
2      2018-01-07 00:00:00
3                    42872
4                    43127
              ...         
195    2017-06-20 00:00:00
196    2018-06-20 00:00:00
197    2017-04-29 00:00:00
198    2019-04-19 00:00:00
199    2019-04-23 00:00:00
Name: 등록일, Length: 200, dtype: object

In [117]:

# 고객 대장의 등록일 오류 수정
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit()
flg_is_serial.sum()

22

* 액셀에서 가져온거라 서식이 안맞음. (엑셀의 경우 날짜를 입력하면 `1900년 1월 1일 이후` 경과된 일수를 출력)
* isdigit() 메소드 : 각 문자열의 **`모든`** 문자가 숫자인지 확인하는 문자열 메소드

In [118]:
pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].astype("float"), unit="D")

1     42782 days
3     42872 days
4     43127 days
21    42920 days
27    42901 days
47    42741 days
49    42929 days
53    42833 days
76    43188 days
80    43110 days
99    42885 days
114   43254 days
118   43129 days
122   43206 days
139   42880 days
143   42818 days
155   42754 days
172   43181 days
179   42743 days
183   42940 days
186   43294 days
192   43259 days
Name: 등록일, dtype: timedelta64[ns]

* timedelta
* 숫자 -> 날짜형식으로

In [119]:
pd.to_datetime("1900/01/01")

Timestamp('1900-01-01 00:00:00')

In [26]:

fromserial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].astype("float"), unit="D") + pd.to_datetime("1900/01/01")
fromserial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

* pd.to_timedelta : 숫자를 날짜로 변경해줌



### ⛔️ 엑셀에서 날짜를 읽어 들일 때 주의할 점

* 엑셀의 날짜 형식의 숫자를 단순히 파이썬으로 계산하면 2일이 어긋남.


* 엑셀 데이터의 출발 숫자는 1부터 VS 파이썬은 0부터
* 엑셀에서 1900/02/29를 유효한 날짜로 계산함(엑셀 버그) VS 파이썬에서는 1900/02/29라는 날짜는 없음.(카운트 자체가 안됨)

In [120]:

fromserial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].astype("float") - 2, unit="D") + pd.to_datetime("1900/01/01")
fromserial

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 등록일, dtype: datetime64[ns]

* 따라서, 엑셀 숫자값에서 -2를 해줘야 정확한 날짜 데이터로 출력.

In [121]:

fromstring = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "등록일"])
fromstring

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [122]:

# 데이터 결합
kokyaku_data["등록일"] = pd.concat([fromserial, fromstring])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-16
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-17
4,김시온,E시,2018-01-27
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [123]:

# 등록일 -> 하던대로 월 추출
kokyaku_data["등록연월"] = kokyaku_data["등록일"].dt.strftime("%Y%m")
rlst = kokyaku_data.groupby("등록연월").count()["고객이름"]
print(rlst)

등록연월
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64


In [124]:
len(kokyaku_data)

200

In [125]:

# 등록일 칼럼에 숫자 데이터가 여전히 남아 있는지 확인
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit()
flg_is_serial.sum()

0

### 018. 고객 이름을 키로 두 개의 데이터를 결합(조인)하자

In [126]:

# 두 개의 데이터의 서로 다른 열을 지정해서 결합 #-> pd.merge 사용 시 left_on, right_on으로 열 이름 따로 지정 
join_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name", right_on="고객이름", how="left")
join_data = join_data.drop("customer_name", axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,201907,김승주,E시,2018-07-13,201807


### 019. 정제한 데이터를 덤프(출력)하자

In [127]:
join_data.columns

Index(['purchase_date', 'item_name', 'item_price', 'purchase_month', '고객이름',
       '지역', '등록일', '등록연월'],
      dtype='object')

In [128]:

dump_date = join_data[['purchase_date', 'purchase_month', 'item_name','item_price',
                       '고객이름', '지역', '등록일', '등록연월']]

* purchase_date와 purchase_month는 가까이 있는 편이 좋음


* 즉, 칼럼 배치 조정 후 파일로 저장하는 것이 직관적으로 이해하기 좋다.

In [129]:

dump_date.to_csv("dump_data.csv", index=False)

* 원본 데이터의 변동이 없는 한 이 csv파일로 분석 가능

### 020. 데이터를 집계하자

In [130]:

import_data = pd.read_csv("dump_data.csv")
import_data

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00,201701
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00,201804
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19 00:00:00,201806
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00,201807
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07 00:00:00,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01 00:00:00,201707
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31 00:00:00,201803
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15 00:00:00,201703
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-13 00:00:00,201807


In [131]:

# purhcase_month를 세로축으로, item_name을 가로축으로
byitem = import_data.pivot_table(index="purchase_month", columns="item_name", 
                                 aggfunc="size", fill_value=0)
byitem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


* 상품이 A~Z까지 정확하게 구입 연월별로 집계됨.

In [132]:

# purhcase_month를 세로축으로, item_name을 가로축으로
# 집계칼럼을 item_price으로
byprice = import_data.pivot_table(index="purchase_month", columns="item_name", 
                                  values="item_price", aggfunc="sum", fill_value=0)
byprice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [133]:

# purhcase_month를 세로축으로, 고객이름을 가로축으로
bycustomer = import_data.pivot_table(index="purchase_month", columns="고객이름", 
                                     aggfunc="size", fill_value=0)
bycustomer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,...,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,...,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,...,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,...,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,...,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,...,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,...,2,1,0,4,0,2,6,1,2,3


In [134]:

# purhcase_month를 세로축으로, 지역을 가로축으로
byregion = import_data.pivot_table(index="purchase_month", columns="지역", 
                                   aggfunc="size", fill_value=0)
byregion

지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [135]:

away_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name", right_on="고객이름", how="right")
away_data[away_data["purchase_date"].isnull()]

,purchase_date,item_name,item_price,customer_name,purchase_month,고객이름,지역,등록일,등록연월
2999,NaT,NaN,NaN,NaN,NaN,김서우,D시,2019-04-23,201904


In [136]:

away_data[away_data["purchase_date"].isnull()][["고객이름", "등록연월"]]

,고객이름,등록연월
2999,김서우,201904


* 집계 기간에 구매 이력이 없는 사용자 확인 -> 즉, 이탈 고객으로 볼 수 있음.